# Radiology Image Classification

#### Import the required libraries

In [ ]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import PIL
import matplotlib.pyplot as plt

from tensorflow import keras
from keras import layers, Input, Model
from keras.models import Sequential
from keras.applications.resnet50 import ResNet50
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
from keras.models import Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

#### Import the datasets

In [ ]:
# Import the dataset
training_dir = 'C:/Users/mming/OneDrive/Desktop/Chest-Classification'
# Windows PWD: C:/Users/mming/OneDrive/Desktop/Chest-Classification
# OSX PWD: /Users/markmingo/Desktop/MURA-v1.1/train
test_dir = 'C:/Users/mming/OneDrive/Desktop/Chest-Classification-Test'
# Windows PWD: C:/Users/mming/OneDrive/Desktop/Chest-Classification-Test
# OSX PWD: 

IMG_HEIGHT = 224
IMG_WIDTH = 224
IMG_SIZE = (IMG_HEIGHT, IMG_WIDTH)
BATCH_SIZE = 32

VALIDATION_TEST_SPLIT = 690

# Create training set
ds_train = keras.utils.image_dataset_from_directory(
    training_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE),
    seed=123,
    validation_split=0.15,
    subset="training"
)

# Create cross-validation set
ds_validation = keras.utils.image_dataset_from_directory(
    training_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE),
    seed=123,
    validation_split=0.15,
    subset="validation"
)

# Create test set
ds_test = keras.utils.image_dataset_from_directory(
    test_dir,
    shuffle=False,
    batch_size=BATCH_SIZE,
    image_size=(IMG_SIZE)
)

# Class names
class_names = ds_train.class_names
print(class_names)

#### Display specific image(s) 

In [ ]:
# Visualize the data
plt.figure(figsize=(10, 10))
image, label = next(iter(ds_train))
for images, labels in ds_train.take(1):
    for i in range(10):
        ax = plt.subplot(5, 2, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis('off')

#### Add data augmentation

In [ ]:
data_augmentation = Sequential([
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2),
    layers.RandomRotation(0.5),
    layers.RandomZoom(-0.1, -0.25),
])

#### Configure dataset's for performance

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

ds_train = ds_train.prefetch(buffer_size=AUTOTUNE)
ds_validation = ds_validation.prefetch(buffer_size=AUTOTUNE)

#### Display an image with data augmentation

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in ds_train.take(1):
    for i in range(9):
        augmented_image = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(augmented_image[0].numpy().astype('uint8'))
        plt.axis('off')

### Convert images from RBG to BGR and zero-center each color channel.

###### Reguired for ResNet50 with ImageNet transfer learning.

In [ ]:
preprocess_input = tf.keras.applications.resnet50.preprocess_input

#### Create the model

In [ ]:
IMG_SHAPE = IMG_SIZE + (3,)

base_model = ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=IMG_SHAPE
)

#### Freeze layers for feature extraction

In [ ]:
base_model.trainable = False

#### Model Summary

In [ ]:
base_model.summary()

In [ ]:
image_batch, label_batch = next(iter(ds_train))
feature_batch = base_model(image_batch)
print(feature_batch.shape)

#### Add a classification head

In [ ]:
global_avg_layer = GlobalAveragePooling2D()
feature_batch_avg = global_avg_layer(feature_batch)
print(feature_batch_avg.shape)

#### Add a prediction layer (multi-class classification)

In [ ]:
prediction_layer = Dense(len(class_names), activation='softmax')
prediction_batch = prediction_layer(feature_batch_avg)
print(prediction_batch.shape)

#### Construct the model

In [ ]:
inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_avg_layer(x)
#x = Dropout(0.2)(x)
outputs = prediction_layer(x)
model = Model(inputs, outputs)

model.summary()


#### Visual plot of the model

In [ ]:
tf.keras.utils.plot_model(model)

#### Compile the model

In [ ]:
alpha = 1e-4
model.compile(
    optimizer=Adam(learning_rate=alpha),
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=[SparseCategoricalAccuracy(name='accuracy')]
)


#### Train the model

In [ ]:
history = model.fit(
    ds_train,
    epochs=10,
    validation_data=ds_validation
)

#### Accuracy: Training & validation

In [ ]:
accuracy = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()), 1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch')
plt.ylim([min(plt.ylim()), 1])
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
print("GPU's: ", tf.config.list_physical_devices('GPU'))